#Animals by the Numbers 2 - Productivity (calculations)

by Karthik Sekar

## Introducton

Herein are my calculations to calculate the maximum growth rate of animals so that they can be easily compared to other domains of life (e.g. plants, microorganisms).

##Preliminary calculations

For calculating the growth rate of animals that grow ontogenetically, we can use the equation from G. B. West, J. H. Brown, and B. J. Enquist, ‘A General Model for Ontogenetic Growth’, Nature, 413.6856 (2001), 628–31

First start with the equation that describes ontogenetic growth (equation 4 from the paper):

$\frac{dm}{dt} =  am^{3/4} \left[ 1 - \left( \frac{m}{M} \right)^{1/4} \right]$

We can get the maximum growth rate, first by differentiating the above equation. Calculation from [here]


$\frac{d^{2}m}{dt^{2}} = \frac{ a \left( 4 (\frac{m}{M})^{1/4} - 3 \right) \frac{dm}{dt}}{4 m^{1/4}}$

Growth rate is maximum when the above equation is equal to 0. Examination suggest that we only have to worry about one term for that to occur, specifically, growth rate is maximum at the following condition:

$0 = \left( 4 (\frac{m}{M})^{1/4} - 3 \right) $

Therefore, at the maximum growth rate, we can say the following:

$(\frac{m}{M})^{1/4} = 3/4$

We can plug back into equation 4 from the paper and find that

$\frac{dm}{dt}_{max} = a \left[ (3/4)^4 M \right]^{3/4} (1-3/4) =  \frac{27}{256} a M^{3/4}$

Good! Almost there. Let's see if this makes sense. Plug in some values for cows and hens. Units are g per day per animal.



In [6]:
import numpy as np

cow_mperday = 27/256*0.276 *np.power(442000,3/4)
print(cow_mperday)

hen_mperday = 27/256*0.502 *np.power(2050,3/4)
print(hen_mperday)

498.998893869
16.1303173479


##Further refinements

The numbers are sensical! We need to normalize by the initial mass of the organisms and convert to per hour (so that it's easy to relate to microbes). I'll use $\mu$ for the growth rate. We finally end with the following:

$\mu = \frac{27}{6144} \frac{a M^{3/4}}{m_0} $

This gives the *maximum* growth rate of any organism that grows ontogenetically. It relies on three parameters that are found in just about every ontogenetic growth paper (a, M, and m0). So how does this compare?

##Cross comparison